<a href="https://colab.research.google.com/github/ThousandAI/pycs4001/blob/main/kaggle_rf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
train_data = pd.read_csv("https://raw.githubusercontent.com/ThousandAI/pycs4001/main/class09/train.csv")
test_data = pd.read_csv("https://raw.githubusercontent.com/ThousandAI/pycs4001/main/class09/test.csv")
test_ids = test_data["PassengerId"]

In [ ]:
def clean(data):
    data = data.drop(["PassengerId", "Name", "Ticket"], axis=1)

    cols = ["SibSp", "Parch", "Fare", "Age"]
    for col in cols:
        data[col].fillna(data[col].mean(), inplace=True)

    data.Embarked.fillna("U", inplace=True)

    return data

In [ ]:
train_data = clean(train_data)
test_data = clean(test_data)
X = train_data.iloc[:, 1:]
Y = train_data.iloc[:, 0]

In [ ]:
concat_data = pd.concat([X, test_data], axis=0)
dummy_data = pd.get_dummies(concat_data)

dummy_train_data = dummy_data.iloc[:891, :]
dummy_test_data = dummy_data.iloc[891:, :]

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
sc_x = scaler.fit_transform(dummy_train_data)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
rf = RandomForestClassifier(max_depth=5)
rf.fit(sc_x, Y)

RandomForestClassifier(max_depth=5)

In [ ]:
depth = []
for i in tqdm(range(2,50,1)):
  rf = RandomForestClassifier(max_depth=i)
  scores = cross_val_score(rf, sc_x, Y, cv=5)
  depth.append(scores.mean())

100%|██████████| 48/48 [01:00<00:00,  1.25s/it]


In [ ]:
depth = []
for i in tqdm(range(50,200,1)):
  rf = RandomForestClassifier(n_estimators=i, max_depth=22)
  scores = cross_val_score(rf, sc_x, Y, cv=5)
  depth.append(scores.mean())

100%|██████████| 150/150 [04:02<00:00,  1.61s/it]


In [ ]:
final_depth = np.argmax(np.array(depth)) + 50
print(final_depth)

92


In [ ]:
final_rf = RandomForestClassifier(n_estimators=200, max_depth=22)

In [ ]:
final_rf.fit(sc_x, Y)

RandomForestClassifier(max_depth=22, n_estimators=200)

In [ ]:
sc_test_data_x = scaler.transform(dummy_test_data)

In [ ]:
submission = final_rf.predict(sc_test_data_x)

In [ ]:
print(submission)

[0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 1 0 1 0 1 0 0 1 0 1 0 1 1 0 0 0 0 1 0 0 1 0
 0 0 1 0 0 0 1 1 0 0 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 0 0
 1 0 0 1 0 1 1 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0
 0 1 0 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 1 0 0 1 0 0 1 0 0 1 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1
 0 1 0 0 0 0 0 1 0 1 0 1 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 1 1 0 0 0 1 0 1 0
 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 1 1 1 1 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 0 0 0 1 0
 0 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0
 0 0 1 0 1 0 0 1 0 0 1]


In [ ]:
submission_data = pd.DataFrame({"PassengerId": test_ids.values, "Survived": submission })
submission_data.to_csv("submission.csv", index=False)